In [ ]:
!pip install streamlit pyngrok

In [ ]:
!pip install ultralytics opencv-python

In [ ]:
%%writefile app.py
import streamlit as st
from PIL import Image
import torch
import cv2
import numpy as np

from ultralytics import YOLO
model = YOLO('YOLOv8_training/sjhacks_yolo_model_fast/weights/best.pt')


def display_image(image, caption=None):
    """Helper to display images in Streamlit with version compatibility."""
    try:
        # Try using the new parameter
        st.image(image, caption=caption, use_container_width=True)
    except TypeError:
        # Fallback for older Streamlit versions
        st.image(image, caption=caption, use_column_width=True)

# Streamlit app layout
st.title('GeoClean SJ')

st.markdown("""
    <style>
    .stApp {
        background-color: #b8e986;
        color: black;
        font-family: 'Helvetica', sans-serif;
    }

    .title {
        text-align: center;
        font-weight: bold;
        font-size: 42px;
        color: #1c1c1c;
        margin-bottom: 0px;
    }

    [data-testid="stExpander"] {
        background: #d3f9b5;
        border: 2px solid #90ee90;
        border-radius: 12px;
        padding: 10px;
    }

    [data-testid="stExpander"] > details > summary {
        color: black;
    }
    [data-testid="stExpander"] > details > summary:hover {
        color: black;
    }

    [data-testid="stTextInput"], [data-testid="stFileUploader"] {
        background-color: #ffffff;
        color: black;
        border: 1px solid #ccc;
        border-radius: 8px;
        padding: 10px;
        font-size: 16px;
    }

    /* Force labels to black */
    label, .stTextInput label, .stFileUploader label {
        color: black !important;
        font-weight: bold;
    }

    /* Force placeholders inside inputs to black */
    input::placeholder {
        color: black;
    }

    # button[kind="primary"] {
    #     background-color: #4CAF50;
    #     color: white;
    #     border-radius: 10px;
    #     padding: 0.75em 1.5em;
    #     font-size: 16px;
    #     border: none;
    #     transition: background-color 0.3s ease;
    # }
    # button[kind="primary"]:hover {
    #     background-color: #45a049;
    # }

    div.stButton > button {
        background-color: #4CAF50 !important;  /* Green background */
        color: white !important;               /* White text */
        border: none !important;
        border-radius: 10px !important;
        padding: 0.75em 1.5em !important;
        font-size: 16px !important;
        font-weight: bold !important;
        transition: background-color 0.3s ease !important;
    }

    div.stButton > button:hover {
        background-color: #45a049 !important;  /* Darker green on hover */
    }

    div.stAlert[data-testid="stAlert-error"] {
        color: black !important; /* text color */
        background-color: #f8d7da !important; /* optional: light red background */
        border: 1px solid #f5c2c7 !important;
    }
    </style>
""", unsafe_allow_html=True)


st.markdown('<h1 class="title">City Cleaning</h1>', unsafe_allow_html=True)

#Image Upload Section
with st.expander("Upload Image"):
    uploaded_image = st.file_uploader("Choose an image...", type="jpg", key="image")

    if uploaded_image is not None:
        img = Image.open(uploaded_image)
        display_image(img, caption='Uploaded Image')
        st.write("Image processed!")

        # YOLOv8 prediction
        img_cv = np.array(img)  # Convert PIL image to OpenCV format
        results = model(img_cv)  # Make predictions with YOLOv8

        # Filter results for confidence > 0.7
        filtered_boxes = []
        filtered_scores = []
        filtered_classes = []

        boxes = results[0].boxes
        for i in range(len(boxes.conf)):
            if boxes.conf[i] > 0.7:
                filtered_boxes.append(boxes.xyxy[i])
                filtered_scores.append(boxes.conf[i])
                filtered_classes.append(boxes.cls[i])

        # Now draw only the filtered boxes on the image
        img_with_boxes = img_cv.copy()
        for bbox, score, cls in zip(filtered_boxes, filtered_scores, filtered_classes):
            bbox = bbox.int().cpu().numpy()
            x1, y1, x2, y2 = bbox
            label = f"Class {int(cls.item())}: {score:.2f}"
            cv2.rectangle(img_with_boxes, (x1, y1), (x2, y2), (255, 0, 0), 2)
            cv2.putText(img_with_boxes, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

        # Display the image with only high-confidence boxes
        display_image(img_with_boxes, caption='Filtered YOLOv8 Predictions (>70% confidence)')

# Coordinates Input Section
with st.expander("Enter Coordinates"):
    coordinates = st.text_input("Enter coordinates (e.g., latitude, longitude)", placeholder="Latitude, Longitude", key="coordinates")
    if coordinates:
        st.write(f"Coordinates entered: {coordinates}")


#Submit button at the bottom
if st.button("Submit"):
    if uploaded_image is not None and coordinates:
        st.success("Submission successful! 🎉")
    elif uploaded_image is None:
        st.error("Please upload an image before submitting.")
    elif not coordinates:
        st.error("Please enter coordinates before submitting.")


In [ ]:
!streamlit cache clear

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')